# Project Euler Problem Set in Python
### Problems: 92, 65, 89, 79

(Solve problems 3, 14, 15, 92, 65, 35, 89, 79, 32, 38, and 46)

## Square digit chains
### Problem #92

A number chain is created by continuously adding the square of the digits in a number to form a new number until it has been seen before.

For example,

&emsp;&emsp; 44 → 32 → 13 → 10 → 1 → 1<br>
&emsp;&emsp; 85 → 89 → 145 → 42 → 20 → 4 → 16 → 37 → 58 → 89

    
Therefore any chain that arrives at 1 or 89 will become stuck in an endless loop. What is most amazing is that EVERY starting number will eventually arrive at 1 or 89.

How many starting numbers below ten million will arrive at 89?

In [55]:
from math import factorial
from itertools import combinations_with_replacement

_SQ_DIG = [d*d for d in range(10)]
_SQ_DIG_SUM = [sum(_SQ_DIG[ord(c) - 48] for c in str(k)) for k in range(1000)]

def square_digit_sum(n):
    sdsum = 0
    while n > 0:
        sdsum += _SQ_DIG_SUM[n % 1000]
        n //= 1000
    return sdsum

def converge(n): 
    while n != 1 and n != 89:
        n = square_digit_sum(n) 
    return n

def count_perms(seq): 
    nperms = factorial(len(seq))
    for d in set(seq):
        nperms //= factorial(seq.count(d))
    return nperms

def square_digit_chains(ndigits, target=89):   
    
    assert 0 < ndigits and (target == 89 or target == 1)
    
    convergeC1 = lambda n: square_digit_sum(n) in C1
    C1 = { n for n in range(1, ndigits*81 + 1) if converge(n) == 1 }
    
    count = 0    
    for cmb in combinations_with_replacement("0123456789", ndigits):
        ns = ''.join(cmb)
        if ns[-1] == '0': continue
        if convergeC1(int(ns)) == 1:
            count += count_perms(ns)
            
    return 10**ndigits-count-1 if target == 89 else count

In [56]:
assert square_digit_chains(7) == 8581146

#### Answer: 8581146
---

## Convergents of e
### Problem 65

The square root of 2 can be written as an infinite continued fraction.

$\sqrt{2} = 1 + \dfrac{1}{2 + \dfrac{1}{2 + \dfrac{1}{2 + \dfrac{1}{2 + ...}}}}$

The infinite continued fraction can be written, $\sqrt{2} = [1; (2)]$ indicates that 2 repeats ad *infinitum*. In a similar way, $\sqrt{23} = [4; (1, 3, 1, 8)]$.

It turns out that the sequence of partial values of continued fractions for square roots provide the best rational approximations. Let us consider the convergents for $\sqrt{2}$.

$
1 + \dfrac{1}{2} = \dfrac{3}{2}\\
1 + \dfrac{1}{2 + \dfrac{1}{2}} = \dfrac{7}{5}\\
1 + \dfrac{1}{2 + \dfrac{1}{2 + \dfrac{1}{2}}} = \dfrac{17}{12}\\
1 + \dfrac{1}{2 + \dfrac{1}{2 + \dfrac{1}{2 + \dfrac{1}{2}}}} = \dfrac{41}{29}
$

Hence the sequence of the first ten convergents for $\sqrt{2}$ are:

$1, \dfrac{3}{2}, \dfrac{7}{5}, \dfrac{17}{12}, \dfrac{41}{29}, \dfrac{99}{70}, \dfrac{239}{169}, \dfrac{577}{408}, \dfrac{1393}{985}, \dfrac{3363}{2378}, ...$

What is most surprising is that the important mathematical constant,

$e = [2; 1, 2, 1, 1, 4, 1, 1, 6, 1, ... , 1, 2k, 1, ...]$

The first ten terms in the sequence of convergents for e are:

$2, 3, \dfrac{8}{3}, \dfrac{11}{4}, \dfrac{19}{7}, \dfrac{87}{32}, \dfrac{106}{39}, \dfrac{193}{71}, \dfrac{1264}{465}, \dfrac{1457}{536}, ...$

The sum of digits in the numerator of the 10<sup>th</sup> convergent is 1 + 4 + 5 + 7 = 17.

Find the sum of digits in the numerator of the 100<sup>th</sup> convergent of the continued fraction for $e$.

In [29]:
from itertools import count, repeat, cycle

sqrt02_denoms = lambda: repeat(2)
sqrt23_denoms = lambda: cycle((1, 3, 1, 8))

def e_denoms():
    for i in count(1):
        yield from (1, 2*i, 1)

In [30]:
from fractions import Fraction as F

def _convergence(denoms, count):
    d = next(denoms)
    if count <= 1: return d
    f = _convergence(denoms, count - 1)
    return d + F(1, f)

def convergence(start, denoms, count=100):
    if count <= 1: return start
    return start +  F(1, _convergence(denoms, count - 1))

In [31]:
from math import sqrt, e, isclose

f = convergence(1, sqrt02_denoms())
assert isclose(f.numerator/f.denominator, sqrt(2))

f = convergence(4, sqrt23_denoms())
assert isclose(f.numerator/f.denominator, sqrt(23))

f = convergence(2, e_denoms())
assert isclose(f.numerator/f.denominator, e)

In [32]:
digit_sum = lambda n: sum(map(int, str(n)))
f = convergence(2, e_denoms())
assert digit_sum(f.numerator) == 272

#### Answer: 272
---

## Roman numerals
### Problem 89

For a number written in Roman numerals to be considered valid there are basic rules which must be followed. Even though the rules allow some numbers to be expressed in more than one way there is always a "best" way of writing a particular number.

For example, it would appear that there are at least six ways of writing the number sixteen:

<pre>
   IIIIIIIIIIIIIIII  
   VIIIIIIIIIII  
   VVIIIIII  
   XIIIIII  
   VVVI  
   XVI  
</pre>

However, according to the rules only XIIIIII and XVI are valid, and the last example is considered to be the most efficient, as it uses the least number of numerals.

The 11K text file, [roman.txt](p089_roman.txt) (right click and 'Save Link/Target As...'), contains one thousand numbers written in valid, but not necessarily minimal, Roman numerals; see About... Roman Numerals for the definitive rules for this problem.

Find the number of characters saved by writing each of these in their minimal form.

Note: You can assume that all the Roman numerals in the file contain no more than four consecutive identical units.

In [237]:
# (empty), I, II, III, IV, V, VI, VII, VIII, IX
DIGIT_LENGTHS = [0, 1, 2, 3, 2, 1, 2, 3, 4, 2]

def roman_numeral_len(n):
    assert 1 < n < 5000
    result = 0
    if n >= 4000: # MMMM
        result += 2  
    while n > 0:
        result += DIGIT_LENGTHS[n % 10]
        n //= 10
    return result

ROMAN_NUMERALS_PREFIXES = [
    ("M" , 1000),
    ("CM",  900),
    ("D" ,  500),
    ("CD",  400),
    ("C" ,  100),
    ("XC",   90),
    ("L" ,   50),
    ("XL",   40),
    ("X" ,   10),
    ("IX",    9),
    ("V" ,    5),
    ("IV",    4),
    ("I" ,    1)]

def parse_roman_numeral(s):
    result = 0
    while len(s) > 0:
        for (prefix, val) in ROMAN_NUMERALS_PREFIXES:
            if s.startswith(prefix):
                result += val
                s = s[len(prefix): ]
                break
        else:
            raise Exception("Cannot parse Roman numeral")
    return result

In [238]:
with open('p089_roman.txt') as f: data = f.read().strip().splitlines()
letter_count = sum(len(s) - roman_numeral_len(parse_roman_numeral(s)) for s in data)

assert letter_count == 743

#### Answer: 743
---

## Passcode derivation
### Problem 79

A common security method used for online banking is to ask the user for three random characters from a passcode. For example, if the passcode was 531278, they may ask for the 2nd, 3rd, and 5th characters; the expected reply would be: 317.

The text file, [keylog.txt](p079_keylog.txt), contains fifty successful login attempts.

Given that the three characters are always asked for in order, analyse the file so as to determine the shortest possible secret passcode of unknown length.

In [239]:
def swap(lst, i, j):
    lst[i], lst[j] = lst[j], lst[i]

In [240]:
with open('p079_keylog.txt') as f: 
    data = f.read().splitlines()

for entry in data:
    # a < b < c 
    a, b, c = entry
    assert b+a not in data
    assert c+b not in data
    assert c+a not in data

digits = {*()}.union(*data)

guess = [*digits]
for rule in data:
    a, b, c = rule
    ia = guess.index(a)
    ib = guess.index(b)
    ic = guess.index(c)
    if ia > ib: swap(guess, ia, ib)
    if ib > ic: swap(guess, ib, ic)
    if ia > ic: swap(guess, ia, ic)

assert ''.join(guess) == '73162890'

#### Answer: 73162890
---